In [53]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVR

In [2]:
tr_df=pd.read_csv('Train-Set.csv')
tr_df

,ProductID,Weight,FatContent,ProductVisibility,ProductType,MRP,OutletID,EstablishmentYear,OutletSize,LocationType,OutletType,OutletSales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [3]:
#Continous cols in dataset
con_cols=[col for col in tr_df.columns if tr_df[col].dtype=='int64' or tr_df[col].dtype=='float64']

In [4]:
#Loop to detect and remove outliers in all cont cols
#for col in con_cols:
 #   q1=tr_df[col].quantile(0.25)
  #  q3=tr_df[col].quantile(0.75)
   # iqr=q3-q1
    #lvo=q1-1.5*iqr
    #hvo=q3+1.5*iqr
    #tr_df_out=tr_df.loc[(tr_df[col]>lvo) & (tr_df[col]< hvo)]

In [5]:
tr_df

,ProductID,Weight,FatContent,ProductVisibility,ProductType,MRP,OutletID,EstablishmentYear,OutletSize,LocationType,OutletType,OutletSales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [6]:
#creating output variable and droping from main DF
y=tr_df['OutletSales']
tr_df.drop(columns='OutletSales',inplace=True)

In [7]:
#Continous cols in dataset
con_cols_new=[col for col in tr_df.columns if tr_df[col].dtype=='int64' or tr_df[col].dtype=='float64']

In [8]:
#cat cols in dataset
cat_cols_new=[col for col in tr_df.columns if tr_df[col].dtype=='object']
cat_cols_new


['ProductID',
 'FatContent',
 'ProductType',
 'OutletID',
 'OutletSize',
 'LocationType',
 'OutletType']

In [9]:
#Train Test Split
x_train,x_test,y_train,y_test=train_test_split(tr_df,y,random_state=42,test_size=0.2)


In [10]:
#Missing Values Replace using Iterative Imputer for cont cols
itr=IterativeImputer(estimator =LinearRegression())
x_train[['Weight']]=itr.fit_transform(x_train[['Weight']])
x_test[['Weight']]=itr.transform(x_test[['Weight']])

In [11]:
#Missing Values Replace using mode in cat cols
for col in cat_cols_new:
    x_train[col].fillna(x_train[col].mode()[0],inplace=True)
    x_test[col].fillna(x_train[col].mode()[0],inplace=True)

In [12]:
#Scaling cont variables
scalar=StandardScaler()

for col in con_cols_new:
    x_train[col]=scalar.fit_transform(np.array(x_train[col]).reshape(-1,1))
    x_test[col]=scalar.transform(np.array(x_test[col]).reshape(-1,1))


In [13]:
#One Hot encoding
cat_encd_train= pd.get_dummies(x_train[cat_cols_new])
cat_encd_test=pd.get_dummies(x_test[cat_cols_new])

In [14]:
#Aliging train data with test data
cat_encd_train_final,cat_encd_test_final=cat_encd_train.align(cat_encd_test,join='inner',axis=1)
cat_encd_train_final.shape###aligning train & test data one hot encoded catg columns due to unqual no of columns i.e no of cilumns would differ for that we align to get same

(6818, 1096)

In [15]:
#concatinating train scaled data & one hot data for both train & test data
x_train_final=pd.concat([x_train[con_cols_new],cat_encd_train_final],axis=1)
x_test_final=pd.concat([x_test[con_cols_new],cat_encd_test_final],axis=1)

In [16]:
x_test_final.shape

(1705, 1100)

In [17]:
from sklearn.tree import DecisionTreeRegressor
dtree_reg=DecisionTreeRegressor(max_depth=5,min_samples_split=10, min_samples_leaf=10)

In [18]:
dtree_reg.fit(x_train_final,y_train)


DecisionTreeRegressor(max_depth=5, min_samples_leaf=10, min_samples_split=10)

In [19]:
y_test_pred=dtree_reg.predict(x_test_final)
y_test_pred


array([1507.07681196,  661.96429679,  661.96429679, ...,  661.96429679,
        661.96429679, 1507.07681196])

In [20]:
dtree_reg.score(x_test_final,y_test)# R2 score for test data

0.6113204488994815

In [21]:
dtree_reg.score(x_train_final,y_train)

0.6006914274684098

In [22]:
from sklearn.metrics import r2_score
r2_score(y_test,y_test_pred)

0.6113204488994815

In [23]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_test_pred)

1056419.3853972342

In [29]:
from sklearn.ensemble import RandomForestRegressor
#Random Forest Regressor bagging technique
rdfc=RandomForestRegressor()
rdfc.fit(x_train_final,y_train)
rdfc_test_pred=rdfc.predict(x_test_final)
rdfc_train_pred=rdfc.predict(x_train_final)

In [30]:
#R2 score of test data using RF
r2_score(y_test,rdfc_test_pred)

0.5610602721789872

In [31]:
#R2 score of train data using RF
r2_score(y_train,rdfc_train_pred)

0.9373703947883987

In [33]:
from sklearn.model_selection import RandomizedSearchCV
# Hyper parameter tuning using Grid serch CV
dict={
    'n_estimators': [300,500,800],
    'max_depth':[5,8],
    'min_samples_split':[5,3],
    'min_samples_leaf':[1,2]
    } 

rsv=RandomizedSearchCV(rdfc,dict,cv=5)

In [34]:
rsv.fit(x_train_final,y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'max_depth': [5, 8],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [5, 3],
                                        'n_estimators': [300, 500, 800]})

In [35]:
rsv.best_params_

{'n_estimators': 500,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_depth': 5}

In [39]:
rsv_test_pred=rsv.predict(x_test_final)
rsv_train_pred=rsv.predict(x_train_final)

In [40]:
#R2 score of test data using RF
r2_score(y_test,rsv_test_pred)

0.6166270614003291

In [42]:
#R2 score of train data using RF
r2_score(y_train,rsv_train_pred)

0.6077635206028957

In [58]:
from sklearn.ensemble import  AdaBoostRegressor
#Ada Boost Regressor technique
adrg=AdaBoostRegressor(base_estimator=LinearRegression(),n_estimators=500)
adrg.fit(x_train_final,y_train)
adrg_test_pred=adrg.predict(x_test_final)
adrg_train_pred=adrg.predict(x_train_final)

In [59]:
#R2 score of test data using AdaBoost Boosting technique
r2_score(y_test,adrg_test_pred)

0.5281397087350339

In [60]:
#R2 score of train data using AdaBoost Boosting technique
r2_score(y_train,adrg_train_pred)


0.568992519575088